Este proyecto se compone de distintos módulos que contienen las diferentes líneas de investigación que se han llevado a cabo para el desarrollo de este cuaderno. 

En las respectivas carpetas se encontrarás archivos `.py` y `jupyter notebooks` junto con sus respectivas carpetas anidadas `data` que contienen los datos utilizados en cada módulo.

# **PIANO SOUL**

## Idea inicial

#### PianoSoul es un proyecto que nace de la idea de querer representar musicalmente (en nuestro caso con un piano) cómo nos sentimos. 
#### El objetivo principal era, que el usuario que utilice PianoSoul, escribiera un texto y este fuera analizado por un LLM. Con esto se obtendrían unas características que posteriormete se le proporcionarían a un modelo y este generaría una melodía, cadencia o una pequeña composición musical para dicho usuario.

#### El objetivo final del proyecto es acercar la música y la composición a personas que no tienen conocimientos teóricos.
#### Para ello utilizamos, el lenguaje natural como fuente para expresar sentimientos que se reflejan en una progresión de acordes, en la que el usuario puede basar una parte de su propia canción, dandole, además, la facilidad de tener resaltadas las notas de la escala para que pueda improvisar a través de la página web.

---

## Desarrollo

### 1. Primer módulo &rightarrow; `project_develp`


Este fue el primer módulo del proyecto que implementamos. 

Partimos de un conjunto de datos sobre fragmentos musicales encontrados en [Kaggle](https://cvml.unige.ch/databases/DEAM/). De estos datos utilizamos el .cvs con datos sobre la positividad y la activación sobre los fragmentos musicales analizados y la carpeta con los respectivos audios (mp3).


Empezamos el proyecto buscando en internet cómo poder etiquetar los datos que teníamos con unos sentimientos de manera objetiva, sin interferir nosotros sesgando las emociones de manera individual.
Encontramos que podíamos definir las emociones **científicamente** basándonos en el modelo de [**circunflejo emocional**](https://psicologiaingresouna.wordpress.com/contenidos/conducta-motivada-y-emocional/emociones/) (Russell, 1980), que mapea emociones en un plano bidimensional con los ejes de **palcer/displacer** (positividad) y **activación/inactividad**.



<p align="center">
  <img src="https://psicologiaingresouna.wordpress.com/wp-content/uploads/2018/01/circumplex-of-russell1.png" alt="Circumplex of Russell">
</p>



#### **Rangos de Valores**
- **Valence** (positividad):
  - \(-1\): Muy negativo.
  - \(0\): Neutral.
  - \(+1\): Muy positivo.
- **Arousal** (activación):
  - \(-1\): Muy calmado.
  - \(0\): Neutral.
  - \(+1\): Muy activo.

#### **Distribución de Emociones en el Espacio**
Basándonos en combinaciones de valence y arousal, podemos asignar emociones más detalladas:

| **Valence**              | **Arousal**                    | **Emotion**       |
|---------------------------|---------------------------------|-------------------|
| Muy positivo > 0.5        | Muy activo > 0.5               | **Excitement**    |
| Muy positivo > 0.5        | Neutral entre 0.0 y 0.5        | **Joy**           |
| Muy positivo > 0.5        | Muy calmado < 0.0              | **Contentment**   |
| Neutral entre 0.0 y 0.5   | Muy activo > 0.5               | **Elation**       |
| Neutral entre 0.0 y 0.5   | Neutral entre 0.0 y 0.5        | **Happiness**     |
| Neutral entre 0.0 y 0.5   | Muy calmado < 0.0              | **Calmness**      |
| Muy negativo entre -0.5 y 0.0 | Muy activo > 0.5            | **Stress**        |
| Muy negativo entre -0.5 y 0.0 | Neutral entre 0.0 y 0.5    | **Satisfaction**  |
| Muy negativo entre -0.5 y 0.0 | Muy calmado < 0.0          | **Relaxation**    |
| Muy negativo < -0.5       | Muy activo > 0.5               | **Fear**          |
| Muy negativo < -0.5       | Neutral entre 0.0 y 0.5        | **Anxiety**       |
| Muy negativo < -0.5       | Muy calmado < 0.0              | **Sadness**       |
| Muy negativo < -0.5       | Muy calmado extremo < -0.5     | **Depression**    |
| Muy negativo < -0.5       | Activación intermedia entre -0.5 y 0.0 | **Anger**      |
| Neutral cualquier valence | Muy inactivo < -0.7            | **Sleep**         |
| Cualquier valor           | Sin clasificación específica   | **Neutral**       |

---

Tras hacer una limpieza de los valores del CSV, creamos una función `map_emotion` a la que le pasamos los datos del dataset y concatenamos las emociones en una nueva columna (puedes encontrar el dataset final usado en el archivo `features_val_ar_emotions.csv` dentro de la carpeta data).

Obtenemos este gráfico de forma visual apra orientarnos:

<br>
<p align="center">
    <img src="project_develp\data\output_sentimientos.png" alt="grafico emociones mapeo" width="800">
</p>
<br>

--- 

Después, buscamos qué librería nos podía servir para analizar audio. Elegimos **librosa** para procesar archivos .mp3, para todo el tema de las características muscicales. Sin embargo, esta librería tenía una imitación, no permitía obtener datos conceptuales como notas o tonalidades y sólo permitía la extracción de datos estructurales como tempo o ritmo. 

A pesar de que esto nos suponía una limitación muy grande, seguimos con el desarrollo teniendo la esperanza de que el modelo predijera bien esos "datos estructurales" para luego pasárselo a un modelo que generase música. 


Para entrenar el modelo encodeamos (utilizando `pd.get_dummies(df['emotion'], prefix='emotion')`) los sentimientos utilizados en la función de mapeo para que tuvieran todo en forma matricial con 1 y 0.
 
Sin embargo, una vez entrenado el modelo, obtenimos un `mse: 53911182.310126126` y un `r2 score: 0.17416793955996174`, lo que supone un resultado **nefasto**. 

Como además las características que nos daba librosa no nos servían mucho para generar música, **decidimos abandonar esta línea del proyecto y explorar otras ramas.**

---

<br>

### 2. Segundo módulo &rightarrow; `obtencion_acordes`

Decimimos crear nosotros nuestra propia base de datos utilizando distintas fuentes porque no había bases de datos que relacionasen progresiones de acoredes con sentimientos como tal.

---

Encontramos la [API de Hooktheory](https://www.hooktheory.com/api/trends/docs), de la que extraímos primero distintas progresiones de acordes y posteriormente las canciones que utilizaban estas progresiones a través de diferentes endpoints y explorando en una estructura de grafo. Primero sacando las progresiones de 1 acorde, después las de 2, 3 y por último 4. 

Entre los problemas que nos encontramos estaba el límite de peticiones, por lo que tuvimos que realizar una especie de reloj para controlarlas. Como hay muchísimas progresiones diferentes implementamos un threshold para quitarnos aquellas menos utilizadas. El problema está en que la API de por sí solo tiene endpoints para progresiones mayores y no nos dimos cuenta hasta mucho más tarde. Es por ello por lo que digamos que el proyecto es realmente una especie de demo que podría escalar mucho más en un futuro, con acordes menores, modos griegos... en cuanto a la armonía como tal.

---

Posteriormente hicimos peticiones a la API de forma similar pero poniendo un límite de páginas por el tiempo de computación al tener que esperar, obteniendo las canciones que utilizaban esas progresiones de acordes teniendo un dataframe de canciones y la progresion que usan en cada parte de su canción. Otro problema es que la base da datos de Hooktheory tampoco es muy muy grande. 

---

Teníamos la parte fácil, las canciones con sus progresiones independientemente de la tonalidad de estas, pero nos faltaba relacionar esto con los sentimientos de alguna forma. 

Encontramos en kaggle un dataset que relacionaba las canciones con distintos tags que había etiquetado la comunidad. Se nos ocurrió la idea de extraer esos tags y hacer una especie de One Hot Encoding, obteniendo un array binario que nos daría una especie de medida sobre que sentimientos refleja la canción. 

Intentamos abordar también las métricas de valence, arousal, danceabilty... pero al final lo teníamos que relacionar de alguna forma con palabras igualmente así que por simplificidad tomamos estas tags para expresar los sentimientos, un total de 101. Para mejorar la efectividad del modelo en un futuro se podrían tener en cuenta estas métricas también.

---

Teníamos al final dos dataset distintos, los juntamos por Artista y por canción después de prepocersarlos obteniendo por tanto un dataset con las canciones, sus progresiones y los sentimientos que expresa. 


Tomamos como X, el array de sentimientos y como Y la progresión a la que le aplicamos un LabelEncoder. Utilizamos un RandomForest como modelo pero podríamos haber utilizado una red neuronal también. Habíamos creado un modelo capaz de predecir a partir de un array con diferentes características una progresión.

---

Entra en juego la parte del front-end


### 3. Tercer módulo &rightarrow; `piano-webpage`


En base a lo que hemos aprendido en **RID** con **HTML** y **CSS** decidimos hacer una pagina web interactiva en la que hubiera un piano como elemento principal. Para hacerlo funcional utilizamos javascript que, obviamente se escapaba de nuestro conocimiento asi que con ayuda de LLMs acabamos desarrollando el Front. Este es un piano que se puede tocar con las teclas del teclado del ordenador y con el ratón. Un campo para ajustar el tempo, un botón de play y stop, y un selector para elegir la tonalidad.

La idea era que cuando el usuario diera al play se tocase la progresión que predijera el modelo y así podría obtener una base sobre la cual crear su propia canción y pudiera improvisar la melodía en base a las notas de la escala que se coloreaban aunque no tuviera ni idea de música.

---

El problema que nos surgió fue que como predecíamos a partir de un array de 1 si el usuario quería que reflejara un sentimiento o 0 si no, o el usuario indicaba las palabras literalmente o no iba a funcionar bien. Se nos ocurrió utilizar la distancia coseno tras aplicar embeddings. Dado el elevado coste computacional que nos iba a suponer eso, surgió la idea de utilizar la API de un LLM como tal. 

Utilizamos uno de [HuggingFace](https://huggingface.co/docs/api-inference/index), para que funcione necesita una key, que es privada de cada uno (en nuestro caso, para el distrute de quien lo pruebe, la dejamos porque HuggingFace si se leakea la desactiva). 

Entonces, en el Front, utilizamos un campo de texto en el que el usuario introduce las caracterísitcas que quiere y el modelo clasifica en base al texto los sentimientos posibles que habíamos definido y devuelve unas probabilidades que después binarizamos. 

---

El último paso ya era unir el **back-end** con el **front-end**.

Solo nos queda mandar este array a nuestro modelo predictor de progresiones en Python desde el Front. Para ello utilizamos una `API Rest`, que definimos con `Flask`, como hemos visto en RID. Y el modelo devuelve al Front la progresión donde se almacena en una variable global y ya se puede tocar la progresión. 

Todo el proceso se puede ver en la consola en el navegador, obteniendo una aplicación totalmente funcional para aquellos que se quieren adentrar en el mundo de la música y crear sus propias canciones sin tener ni idea simplemente a partir de lo que uno quiere expresar. 

### 4. Cuarto módulo &rightarrow; `api_database`


Esta es la última línea de desarrollo que hemos seguido.

Tras ver el potencial que tiene, para nuestro objetivo, la página web, decidimos emplear un **Modelo de Lenguaje de Largo alcance** para procesar el texto del usuario de `PianoSoul` con un enfoque algo distinto, de forma que no solo le devuelva una progresión, sino que complemte esta con otros factores relevantes a la hora de componer música (sin entrar en características técnicas). 

Implementamos una API personalizada  de [**GoogleAi**](https://ai.google.dev/gemini-api/docs/quickstart?hl=es-419&lang=python) con unos conocimientos adquiridos previamente diseñado por nosotros.

---

Construimos una base de datos en formato `json` con la información de distintas páginas webs que hemos buscado y leído. 

Con la ayuda de librerías como `BeautifulSoup`, obtenemos el contenido de dentro de las etiquetas HTML constuirlo de manera más estructurada. 

Guardamos este texto en formato json. A cada "valor del diccionario" le aplicamos un proceso de preprocesado con `regex` para eliminar urls, etiquetas html, caracteres no alfanuméricos y espacios innecesarios. Mantenemos los siggnos de puntuación para procesar un contexto completo.

Como no encontramos nada muy explícito acerca de los modos musicales, le pedimos a chatgpt que nos hardcodeara una pequeña base de conocimientos sobre modos musicales y sentimientos. Esto también se lo pasaremso a la API

---

Para pasar al proceso de construcción de la API usamos la guía básica de googleai proporiconada en el enlace. 

Nos sacamso nuestra Api key (única e intrasferible) y pasamos a diseñar el prompt que posteriormente procesará.

Para meter en contexto a la IA, le indicamos que actúe como un experto en música y psicología especializada en la relación entre música y emociones. Su tarea es analizar un texto proporcionado por el usuario, comprender el estado emocional subyacente y traducirlo en características musicales específicas, proporcionando un dictado detallado con valores útiles para componer música.

Estas **características musicales** las hemos decidido bajo nuestro criterio de 'personas que tocan un isntrumento' relacionándolo también con lo leído en las páginas web. Son las siguientes: **Tempo, Intensidad/Dinámica, Timbre, Ritmo, Progresión armónica, Melodía, Tonalidad/Modo, Articulación, Silencio.**

---

En el respectivo notebook `api_devel.ipynb` podrás encontrar las respuestas obtenidas a nuestro "user texts" e incluso probarlo tú mismo.

Nosotros hemso probado como sonaba en el piano y nos ha parecido suficientemente acertado para tratarse de algo tan básico.

Esto nos abre nuevas líneas de investigación que quedan como futuras ramas abiertas a continuar.

<br>

---

<br>

## Futuras líneas de investigación:

### - Diseñar nuestro propio LLM 
&nbsp; y entrenarlo para darle un enfoque más ajustado.
### - Desarrollar un modelo basado en redes de neuronas 
&nbsp; que mejores el RandomForest que hemos usado.
### - Mapear el sentimiento de los lyrics del dataset con las canciones y cadencias 
&nbsp; para así obtener un conjunto de datos etiquetados basado en análisis de sentimientos en textos.
### - Enlazar la respuesta de la API con el piano interactivo
&nbsp; para así poder reproducir la cadencia en la tonalidad deseada en el piano y poder inprovisar una pequeña melodía o probar la generada por la api

---

## Conclusiones

En primer lugar, queremos dejar claro que, es imposible generalizar cualquier tipo de música asociando progresiones a sentimientos. 

Para empezar los sentimientos son algo bastante subjetivo y complejo, una misma canción puede generar distintos sentimietnos en cada persona, incluso distintas partes de la canción generan distintas emociones. 

Es por ello que lo hemos intentado simplificar de alguna forma porque, bien es cierto, que distintas progresiones son mas alegres, más tristes...  también depende de diversos factores, como el ritmo, las inversiones, los sonidos, el estilo de música...  esta simplificación puede ser útil para aquellas personas que se quieren adentrar en este mundillo.


Lo que pretendemos ensalzar en este proyecto, es lo que puede conseguir una persona con ayuda de la IA en la música. 

Al final, creemos que la música es una parte de la esencia humana, y sería una cuestión para reflexionar si una IA como Suno, por ejemplo, puede hacer música que transmita esos sentimientos igual que lo hace la música humana.

Nosotros hemos querido mantener esa esencia simplemente dando un empujoncito para empezar en este camino de la composición a cualquier persona.